In [1]:
import os
import cv2
import random
import numpy as np
from skimage.metrics import structural_similarity as ssim

In [2]:
def calculate_psnr_pair(original, reconstructed):
    # Calculate PSNR for a single pair of images.

    # Ensure images are in float32 format for calculations
    original = original.astype(np.float32) / 255.0
    reconstructed = reconstructed.astype(np.float32) / 255.0

    mse = np.mean((original - reconstructed) ** 2)
    if mse == 0:  # Perfect match
        return float('inf')
    max_pixel = 1.0  # Adjust max_pixel for normalized range
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def calculate_nc_pair(original, reconstructed):
    """
    Calculate Normalized Correlation (NC) for a single pair of images.
    Args:
        original (np.ndarray): Original image (ground truth).
        reconstructed (np.ndarray): Watermark-removed image.
    Returns:
        float: NC value between 0 and 1.
    """
    # Normalize pixel values to [0, 1]
    original = original.astype(np.float32) / 255.0
    reconstructed = reconstructed.astype(np.float32) / 255.0

    numerator = np.sum(original * reconstructed)
    denominator = np.sqrt(np.sum(original ** 2) * np.sum(reconstructed ** 2))
    return numerator / denominator if denominator != 0 else 0

def calculate_ssim_pair(original, reconstructed):
    """
    Calculate Structural Similarity Index (SSIM) for a single pair of images.
    Args:
        original (np.ndarray): Original image (ground truth).
        reconstructed (np.ndarray): Watermark-removed image.
    Returns:
        float: SSIM value between -1 and 1 (1 indicates perfect similarity).
    """
    # Ensure images are grayscale
    if len(original.shape) == 3:
        original = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    if len(reconstructed.shape) == 3:
        reconstructed = cv2.cvtColor(reconstructed, cv2.COLOR_BGR2GRAY)
    
    return ssim(original, reconstructed, data_range=reconstructed.max() - reconstructed.min())

def test_pairwise_metrics(original_path, reconstructed_path):
    # Computing PSNR, NC, and SSIM calculations on a single image pair.

    original = cv2.imread(original_path, cv2.IMREAD_GRAYSCALE)
    reconstructed = cv2.imread(reconstructed_path, cv2.IMREAD_GRAYSCALE)
    if original is None or reconstructed is None:
        raise ValueError("Error loading one or both images.")

    psnr = calculate_psnr_pair(original, reconstructed)
    nc = calculate_nc_pair(original, reconstructed)
    ssim_value = calculate_ssim_pair(original, reconstructed)

    print(f"Metrics for the selected image pair:")
    print(f"Original: {original_path}")
    print(f"Reconstructed: {reconstructed_path}")
    print(f"PSNR: {psnr:.2f} dB")
    print(f"NC: {nc:.4f}")
    print(f"SSIM: {ssim_value:.4f}")

def get_random_image_pair(original_dir, reconstructed_dir):
    # Ensure only files that exist in both directories are paired
    original_files = set(sorted(os.listdir(original_dir)))
    reconstructed_files = set(sorted(os.listdir(reconstructed_dir)))
    common_files = sorted(original_files & reconstructed_files)  # Only common files

    if not common_files:
        raise ValueError("No common files found between directories.")

    # Pick a random file from the common set
    random_file = random.choice(common_files)
    original_path = os.path.join(original_dir, random_file)
    reconstructed_path = os.path.join(reconstructed_dir, random_file)
    return original_path, reconstructed_path

In [3]:
# Usage: Comparing Random Image Pairs
if __name__ == "__main__":
    original_dir = "/home/jovyan/fresh_clean_dataset/data/resized_images/test"  # REPLACE and POINT to ORIGINAL images folder
    #reconstructed_dir = "/home/jovyan/fresh_clean_dataset/data/images_cleaned/trainval"  # REPLACE and POINT to RECONSTRUCTED images folder
    reconstructed_dir = "/home/jovyan/ML2 New/watermark_removal_experiments/large_unet/images_cleaned_10r_large_model/test"  # REPLACE and POINT to RECONSTRUCTED images folder
    
    original_path, reconstructed_path = get_random_image_pair(original_dir, reconstructed_dir)
    test_pairwise_metrics(original_path, reconstructed_path)

Metrics for the selected image pair:
Original: /home/jovyan/fresh_clean_dataset/data/resized_images/test/newfoundland_38.jpg
Reconstructed: /home/jovyan/ML2 New/watermark_removal_experiments/large_unet/images_cleaned_10r_large_model/test/newfoundland_38.jpg
PSNR: 34.99 dB
NC: 0.9996
SSIM: 0.9686


In [5]:
# For DATASET WIDE AVERAGES

def calculate_average_metrics(original_dir, reconstructed_dir):

    def filter_images(file_list):
        valid_extensions = (".jpg", ".jpeg", ".png", ".bmp", ".tiff")
        return [f for f in file_list if f.lower().endswith(valid_extensions)]

    # Filter files to include only valid images
    original_files = sorted(filter_images(os.listdir(original_dir)))
    reconstructed_files = sorted(filter_images(os.listdir(reconstructed_dir)))

    if not original_files or not reconstructed_files:
        raise ValueError("One or both directories are empty or contain no valid image files.")

    psnr_values = []
    nc_values = []
    ssim_values = []  # NEW: List to store SSIM values

    for original_file, reconstructed_file in zip(original_files, reconstructed_files):
        original_path = os.path.join(original_dir, original_file)
        reconstructed_path = os.path.join(reconstructed_dir, reconstructed_file)

        original = cv2.imread(original_path, cv2.IMREAD_GRAYSCALE)
        reconstructed = cv2.imread(reconstructed_path, cv2.IMREAD_GRAYSCALE)
        if original is None or reconstructed is None:
            print(f"Warning: Could not load pair ({original_path}, {reconstructed_path}). Skipping...")
            continue

        # Reusing functions
        psnr = calculate_psnr_pair(original, reconstructed)
        nc = calculate_nc_pair(original, reconstructed)
        ssim_value = calculate_ssim_pair(original, reconstructed)  # NEW: Calculate SSIM

        psnr_values.append(psnr)
        nc_values.append(nc)
        ssim_values.append(ssim_value)  # NEW: Append SSIM value

    avg_psnr = np.mean(psnr_values) if psnr_values else 0
    avg_nc = np.mean(nc_values) if nc_values else 0
    avg_ssim = np.mean(ssim_values) if ssim_values else 0  # NEW: Calculate average SSIM

    return avg_psnr, avg_nc, avg_ssim


# Usage for Dataset-Wide Averages
if __name__ == "__main__":
    original_dir = "/home/jovyan/fresh_clean_dataset/data/resized_images/test"  # REPLACE and POINT to ORIGINAL images folder
    #reconstructed_dir = "/home/jovyan/fresh_clean_dataset/data/images_cleaned/test"  # REPLACE and POINT to RECONSTRUCTED images folder
    reconstructed_dir = "/home/jovyan/ML2 New/precise_transparency_models_data/large_unet_0.65/images_cleaned_10r0.65/test"

    print("\nCalculating average metrics across all image pairs...")
    avg_psnr, avg_nc, avg_ssim = calculate_average_metrics(original_dir, reconstructed_dir)
    print("\nOverall Metrics for the Dataset:")
    print(f"Average PSNR: {avg_psnr:.2f} dB")
    print(f"Average NC: {avg_nc:.4f}")
    print(f"Average SSIM: {avg_ssim:.4f}")


Calculating average metrics across all image pairs...

Overall Metrics for the Dataset:
Average PSNR: 31.35 dB
Average NC: 0.9980
Average SSIM: 0.9364
